# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [28]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv('../output/Cities.csv')
vacation_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bethel,41.3712,-73.4140,56.28,77,90,0.00,US,1636511626
1,Hasaki,35.7333,140.8333,64.80,52,100,23.96,JP,1636511897
2,Geraldton,-28.7667,114.6000,72.12,35,40,16.11,AU,1636512054
3,Kapaa,22.0752,-159.3190,80.92,68,20,5.75,US,1636511938
4,Vaini,-21.2000,-175.2000,82.56,89,75,9.22,TO,1636511864
...,...,...,...,...,...,...,...,...,...
548,Dubrovnik,42.6481,18.0922,64.04,48,0,12.66,HR,1636512279
549,Mandalgovi,45.7625,106.2708,29.84,42,26,7.52,MN,1636512146
550,São Gabriel da Cachoeira,-0.1303,-67.0892,73.15,99,100,2.19,BR,1636512279
551,Turbat,26.0023,63.0440,72.07,28,0,12.30,PK,1636512280


In [3]:
print(vacation_df.dtypes)

City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]
# .astype(float)


humidity = vacation_df["Humidity"]
# .astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#condensing to fit weather conditions
weather_conditions_df = vacation_df.loc[(vacation_df["Max Temp"] >= 70)]
weather_conditions_df = vacation_df.loc[(vacation_df["Max Temp"] <= 80)]
weather_conditions_df = vacation_df.loc[(vacation_df["Wind Speed"] <= 10)]
weather_conditions_df = vacation_df.loc[(vacation_df["Cloudiness"] == 0)]


weather_conditions_df



,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Ushuaia,-54.8000,-68.3000,62.26,31,0,5.75,AR,1636511786
21,Cape Town,-33.9258,18.4232,58.12,77,0,4.61,ZA,1636511878
28,Filingué,14.3521,3.3168,84.22,17,0,3.40,NE,1636512063
29,Port Elizabeth,-33.9180,25.5701,50.29,93,0,5.75,ZA,1636511917
30,Busselton,-33.6500,115.3333,64.29,42,0,4.65,AU,1636511864
...,...,...,...,...,...,...,...,...,...
508,Jumla,29.2747,82.1838,56.39,22,0,1.48,NP,1636512260
528,Shangqiu,34.4500,115.6500,52.97,41,0,7.16,CN,1636512269
532,Pingliang,35.5392,106.6861,41.09,16,0,6.67,CN,1636512271
548,Dubrovnik,42.6481,18.0922,64.04,48,0,12.66,HR,1636512279


In [7]:
#dropping null values
weather_conditions_df = weather_conditions_df.dropna()
weather_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,Ushuaia,-54.8000,-68.3000,62.26,31,0,5.75,AR,1636511786
21,Cape Town,-33.9258,18.4232,58.12,77,0,4.61,ZA,1636511878
28,Filingué,14.3521,3.3168,84.22,17,0,3.40,NE,1636512063
29,Port Elizabeth,-33.9180,25.5701,50.29,93,0,5.75,ZA,1636511917
30,Busselton,-33.6500,115.3333,64.29,42,0,4.65,AU,1636511864
49,Necochea,-38.5473,-58.7368,49.84,91,0,10.20,AR,1636512070
52,Khorosheve,49.8528,36.2195,28.72,68,0,8.99,UA,1636512071
61,Carnarvon,-24.8667,113.6333,73.47,46,0,20.71,AU,1636511879
75,Caceres,39.5220,-6.3748,51.80,66,0,3.89,ES,1636512079
101,Sakakah,29.9697,40.2064,56.77,18,0,3.06,SA,1636512089


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#storing variable into hotel_df
hotel_df = weather_conditions_df

#checked to make sure it worked
# hotel_df.head()

#added Hotel Name column
hotel_df["Hotel Name"] = " "
hotel_df.head()

C:\Users\erika\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Ushuaia,-54.8000,-68.3000,62.26,31,0,5.75,AR,1636511786,
21,Cape Town,-33.9258,18.4232,58.12,77,0,4.61,ZA,1636511878,
28,Filingué,14.3521,3.3168,84.22,17,0,3.40,NE,1636512063,
29,Port Elizabeth,-33.9180,25.5701,50.29,93,0,5.75,ZA,1636511917,
30,Busselton,-33.6500,115.3333,64.29,42,0,4.65,AU,1636511864,


In [18]:
# find hotels with parametrs
hotel_list = []

target_type = "lodging"
radius = 5000
key = g_key


# rewrite params dict

params = {
    "types": target_type,
    "radius": radius,
    "key": g_key
}

# Build URL using the Google Maps API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
#     name = row["City"]
    
    params["location"]= f"{lat},{lng}"

    response = requests.get(url, params=params)
    
    response = response.json()
    
#     print(response["results"][0]["name"])
    
    #Save the hotel name to the dataframe
    try:
        print(response["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Not found")
        print("_"*25)

    print("Search complete")
    
# hotel_df 

Albatros Hotel
Search complete
Southern Sun Waterfront Cape Town
Search complete
Not found
_________________________
Search complete
39 On Nile Guest House
Search complete
Observatory Guest House
Search complete
dyd -mar Hotel
Search complete
White beach
Search complete
Hospitality Carnarvon
Search complete
Ahc Hoteles
Search complete
Fakhamat Al Orjoana Apartment
Search complete
Hotel Santa Cruz
Search complete
Die Lemoenhuis
Search complete
Casa Laguna, Bed & Breakfast
Search complete
Baghdad Hotel
Search complete
Caravan Sarai
Search complete
Diego De Almagro Coyhaique
Search complete
Hotel Star inn Peniche
Search complete
Hotel Castrum Villae by Walk Hotels
Search complete
Hotel San Isidro Oficial
Search complete
Wangjiang Building
Search complete
Laoyutang Resort
Search complete
Beatty Lodge
Search complete
Not found
_________________________
Search complete
Don Eddie's Sport Fishing Center
Search complete
Not found
_________________________
Search complete
Not found
_____________

In [19]:
#checked to make sure they got added to the table
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
5,Ushuaia,-54.8000,-68.3000,62.26,31,0,5.75,AR,1636511786,Albatros Hotel
21,Cape Town,-33.9258,18.4232,58.12,77,0,4.61,ZA,1636511878,Southern Sun Waterfront Cape Town
28,Filingué,14.3521,3.3168,84.22,17,0,3.40,NE,1636512063,
29,Port Elizabeth,-33.9180,25.5701,50.29,93,0,5.75,ZA,1636511917,39 On Nile Guest House
30,Busselton,-33.6500,115.3333,64.29,42,0,4.65,AU,1636511864,Observatory Guest House
49,Necochea,-38.5473,-58.7368,49.84,91,0,10.20,AR,1636512070,dyd -mar Hotel
52,Khorosheve,49.8528,36.2195,28.72,68,0,8.99,UA,1636512071,White beach
61,Carnarvon,-24.8667,113.6333,73.47,46,0,20.71,AU,1636511879,Hospitality Carnarvon
75,Caceres,39.5220,-6.3748,51.80,66,0,3.89,ES,1636512079,Ahc Hoteles
101,Sakakah,29.9697,40.2064,56.77,18,0,3.06,SA,1636512089,Fakhamat Al Orjoana Apartment


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
locations.dtypes

Lat    float64
Lng    float64
dtype: object

In [26]:


# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…